# SMOTE Experiment - Address Class Imbalance

Test whether SMOTE (Synthetic Minority Over-sampling Technique) improves F1 score

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

## 1. Load Data

In [ ]:
X_train = pd.read_pickle('../data/features/X_train_temporal.pkl')
X_test = pd.read_pickle('../data/features/X_test_temporal.pkl')
y_train = pd.read_pickle('../data/features/y_train_cls_temporal.pkl')
y_test = pd.read_pickle('../data/features/y_test_cls_temporal.pkl')

print(f"Train: {X_train.shape}")
print(f"Test: {X_test.shape}")
print(f"\nClass distribution (train):")
print(y_train.value_counts())
print(f"\nImbalance ratio: {y_train.value_counts()[0] / y_train.value_counts()[1]:.2f}:1")

## 2. Baseline (No SMOTE)

In [ ]:
print("=" * 60)
print("BASELINE - No SMOTE")
print("=" * 60)

# Train with class_weight='balanced' (current best approach)
model_baseline = LogisticRegression(
    max_iter=1000,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

model_baseline.fit(X_train, y_train)

# Predict with optimized threshold
y_pred_proba = model_baseline.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= 0.54).astype(int)  # Use optimized threshold

baseline_acc = accuracy_score(y_test, y_pred)
baseline_prec = precision_score(y_test, y_pred)
baseline_rec = recall_score(y_test, y_pred)
baseline_f1 = f1_score(y_test, y_pred)
baseline_roc = roc_auc_score(y_test, y_pred_proba)

print(f"\nAccuracy:  {baseline_acc:.4f}")
print(f"Precision: {baseline_prec:.4f}")
print(f"Recall:    {baseline_rec:.4f}")
print(f"F1 Score:  {baseline_f1:.4f}")
print(f"ROC-AUC:   {baseline_roc:.4f}")

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

## 3. With SMOTE

In [ ]:
print("\n" + "=" * 60)
print("WITH SMOTE")
print("=" * 60)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(f"\nOriginal training set: {X_train.shape}")
print(f"Resampled training set: {X_train_resampled.shape}")
print(f"\nClass distribution after SMOTE:")
print(pd.Series(y_train_resampled).value_counts())

# Train on resampled data (no class_weight needed now)
model_smote = LogisticRegression(
    max_iter=1000,
    random_state=42,
    n_jobs=-1
)

model_smote.fit(X_train_resampled, y_train_resampled)

# Predict with same threshold for fair comparison
y_pred_proba_smote = model_smote.predict_proba(X_test)[:, 1]
y_pred_smote = (y_pred_proba_smote >= 0.54).astype(int)

smote_acc = accuracy_score(y_test, y_pred_smote)
smote_prec = precision_score(y_test, y_pred_smote)
smote_rec = recall_score(y_test, y_pred_smote)
smote_f1 = f1_score(y_test, y_pred_smote)
smote_roc = roc_auc_score(y_test, y_pred_proba_smote)

print(f"\nAccuracy:  {smote_acc:.4f}")
print(f"Precision: {smote_prec:.4f}")
print(f"Recall:    {smote_rec:.4f}")
print(f"F1 Score:  {smote_f1:.4f}")
print(f"ROC-AUC:   {smote_roc:.4f}")

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_smote))

## 4. Comparison

In [ ]:
print("\n" + "=" * 60)
print("COMPARISON: Baseline vs SMOTE")
print("=" * 60)

comparison = pd.DataFrame({
    'Baseline (class_weight)': [
        f"{baseline_acc*100:.2f}%",
        f"{baseline_prec*100:.2f}%",
        f"{baseline_rec*100:.2f}%",
        f"{baseline_f1*100:.2f}%",
        f"{baseline_roc*100:.2f}%"
    ],
    'SMOTE': [
        f"{smote_acc*100:.2f}%",
        f"{smote_prec*100:.2f}%",
        f"{smote_rec*100:.2f}%",
        f"{smote_f1*100:.2f}%",
        f"{smote_roc*100:.2f}%"
    ],
    'Change': [
        f"{(smote_acc - baseline_acc)*100:+.2f}",
        f"{(smote_prec - baseline_prec)*100:+.2f}",
        f"{(smote_rec - baseline_rec)*100:+.2f}",
        f"{(smote_f1 - baseline_f1)*100:+.2f}",
        f"{(smote_roc - baseline_roc)*100:+.2f}"
    ]
}, index=['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC-AUC'])

print("\n", comparison)

# Determine winner
print("\n" + "=" * 60)
if smote_f1 > baseline_f1:
    print(f"✅ SMOTE WINS! F1 improved by {(smote_f1 - baseline_f1)*100:.2f} points")
    print(f"   New best F1: {smote_f1*100:.2f}%")
elif smote_f1 < baseline_f1:
    print(f"❌ SMOTE WORSE. F1 decreased by {(baseline_f1 - smote_f1)*100:.2f} points")
    print(f"   Stick with class_weight='balanced' (F1={baseline_f1*100:.2f}%)")
else:
    print("⚖️  TIE. Both approaches perform equally.")
print("=" * 60)

## 5. Recommendation

In [ ]:
print("\n💡 RECOMMENDATION:")
print("-" * 60)

if smote_f1 > baseline_f1:
    print("Use SMOTE for final model.")
    print(f"Training set will be: {X_train_resampled.shape[0]} samples (balanced)")
    print("Remove class_weight='balanced' when using SMOTE.")
else:
    print("Stick with class_weight='balanced' (simpler, no synthetic samples).")
    print(f"Training set remains: {X_train.shape[0]} samples (original)")
    print("SMOTE didn't provide benefit in this case.")

print(f"\nOptimal threshold: 0.54")
print(f"Expected F1 on test set: {max(baseline_f1, smote_f1)*100:.2f}%")